# Análise dos Algoritmos SVM, MLP, KNN e Random Forest baseados na classificação do Dataset "Fetal Health Classification"

In [ ]:
# Bibliotecas básicas
import numpy as np 
import pandas as pd 

# Biblioteca para marcar os tempos de execução
import time as tm

# Biblioteca para separar os dados em treino e teste
from sklearn.model_selection import train_test_split

# Biblioteca de métricas de análise de dados
from sklearn.metrics import f1_score

# Bibliotecas de Análises Estatísticas e Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# Biblioteca para detalhar os resultados obtidos
from sklearn.metrics import classification_report

# Bibliotecas de Aprendizado de Máquina
from sklearn import svm
from sklearn.neural_network import MLPClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Biblioteca para balanceamento de datasets
from imblearn.over_sampling import RandomOverSampler

O dataset utilizado na análise foi o [fetal-health-classification](https://www.kaggle.com/andrewmvd/fetal-health-classification), que se trata de uma amostra resultante da análise de cardiotocografias realizadas em gestantes para conseguir classificar a saúde de seus fetos em "**Normal**", "**Suspeito**" e "**Patológico**". 

In [ ]:
# Carregamos o dataset escolhido
dataset = pd.read_csv("../input/fetal-health-classification/fetal_health.csv")

O dataset possui 22 colunas, sendo 21 atributos, e a última coluna contendo a classificação da amostra, e 2126 linhas, como podemos visualizar abaixo:

In [ ]:
dataset.head()

Após o carregamento do dataset, verificamos se o mesmo possui [**missing values**](https://www.kaggle.com/alexisbcook/missing-values):

In [ ]:
dataset.isnull().sum()

De acordo com a tabela acima, podemos notar que todos os dados estão preenchidos.

A segunda análise necessária nos dados, é verificar a quantidade de amostras de cada uma das classes, para sabermos se o conjunto de dados está balanceado, evitando assim de enviezarmos o algoritmo, que irá priorizar a classe majoritária a fim de melhorar seu score.

In [ ]:
# Método que imprime a quantidade de amostras por classe
def print_number_of_elements_per_class(y):
    classes_name = ['Normal', 'Suspeito', 'Patológico']
    
    classes_id = list(y)
    number_of_elements = [classes_id.count(1), classes_id.count(2), classes_id.count(3)]
    
    print(classes_name[0], number_of_elements[0])
    print(classes_name[1], number_of_elements[1])
    print(classes_name[2], number_of_elements[2])
    
    plt.bar(classes_name, number_of_elements)
    plt.show()

In [ ]:
# Dividimos os atributos (X) da classificação (y) das amostras
X = np.array(dataset.drop(columns = ['fetal_health']))
y = np.array(dataset['fetal_health'])

print_number_of_elements_per_class(y)

Observando o gráfico obtido, nota-se que os dados estão completamente desbalanceados, a classe "Normal" possui muito mais amostras que as demais, sendo assim, antes de executarmos nossos algoritmos de classificação, iremos aplicar a técnica de [**Oversampling**](https://en.wikipedia.org/wiki/Oversampling) no dataset para balancear os dados.

In [ ]:
X, y = RandomOverSampler(random_state = 21).fit_resample(X,y)

print_number_of_elements_per_class(y)

Depois de realizado o balanceamento, temos quantidades iguais de amostras por classe, o que permite obter um melhor resultado na execução de nossos algitmos classificadores.

A próxima etapa é dividir nossos dados em treino e teste, a divisão escolhida foi de 70% para treino (1159) e 30% para teste (496).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 19, stratify = y)

Para visualizar como os dados estão dividos e se seguem algum padrão, podemos plotar seus atributos dois a dois:

In [ ]:
sns.pairplot(dataset[np.array(dataset.columns)], hue='fetal_health')

Tendo ajustado o dataset, podemos gerar nossos classificadores e executar os algoritmos selecionados:

In [ ]:
# Criamos arrays que irão armazenar os tempos de execução e acurácia de cada um dos algoritmos
accuracies, times = [], []

# Também armazenamos os nomes dos algoritmos para utilizar posteriormente na impressão dos resultados
algorithms = ['SVM', 'MLP', 'KNN', 'Random Forest']

# Classificações possíveis dos dados
classes = ['Normal', 'Suspeito', 'Patológico']

# Geramos nossos 4 classificadores com auxílio das bibliotecas importadas
clf_svm = svm.SVC(kernel = 'poly', C = 1)
clf_mlp = MLPClassifier(random_state = 1, learning_rate_init = 0.003 , max_iter = 10000)
clf_knn = KNeighborsClassifier(n_neighbors = 1)
clf_rfc = RandomForestClassifier(random_state = 30)

classifiers = [clf_svm, clf_mlp, clf_knn, clf_rfc]

Após gerados os classificadores, podemos realizar o fit e imprimir o tempo de execução de cada um deles:

In [ ]:
# Imprime os tempos de execução obtidos
def print_exec_times(algorithms):
    for i in range(len(algorithms)):
        print(algorithms[i], " - Tempo de execução em segundos: ", times[i])
        
# Executam o fit dos classificadores
def classifiers_fit(classifiers):
    for i in range(len(classifiers)):  
        init = tm.time()
        classifiers[i].fit(X_train, y_train)
        end = tm.time()

        times.insert(0, end - init)

In [ ]:
classifiers_fit(classifiers)
print_exec_times(algorithms)

Agora que obtemos os classificadores e realizamos o fit de cada um deles, vamos calcular as acurácias:

In [ ]:
# Calcula a acurácia de cada um dos classificadores
def calculate_accuracies(classifiers):
    i = 0
    for classifier in classifiers:
        accuracies.insert(i, classifier.score(X_test, y_test).round(4) * 100)
        i += 1

# Imprime a acurácia de cada um dos classificadores 
def print_accuracies(accuracies, algorithms):
    for i in range(len(classifiers)):
        print(algorithms[i], " - Acurácia: ", accuracies[i], "%")

In [ ]:
calculate_accuracies(classifiers)
print_accuracies(accuracies, algorithms)

Podemos notar que o melhor desempenho foi o do Random Forest, seguido do KNN, os dois quase alcançaram 100% de acurácia.

Como era de se esperar, o MLP foi o algoritmo que obteve a menor quantidade de acertos, pois este algoritmo performa melhor em problemas não-linearmente separáveis.

Agora, vamos imprimir o detalhamento dos resultados obtidos por cada um dos algoritmos:

In [ ]:
# Imprime o report da classificação do algoritmo
def generate_classification_report(X, y, classifier, classes, algorithm): 
    print(algorithm, " - Resultados: \n\n", classification_report(y, classifier.predict(X), target_names=classes))

In [ ]:
for i in range(len(algorithms)):
    generate_classification_report(X_test, y_test, classifiers[i], classes, algorithms[i])

Posteriormente, iremos calcular o f1-score de cada um dos algoritmos:

In [ ]:
f1_scores = []

def generate_classifiers_f1_score(X, y, classifiers, algorithms): 
    for i in range(len(algorithms)):
        f1_score_i = f1_score(y, classifiers[i].predict(X), average='macro')
        f1_scores.insert(i, f1_score_i)
        print(algorithms[i], ": f1-score = ", f1_score_i)

In [ ]:
generate_classifiers_f1_score(X, y, classifiers, algorithms)

Ao final, geramos os gráficos de comparação de tempo de execução, acurácia e f1-score:

In [ ]:
# Geramos o gráfico Algoritmo x Tempo de Execução
plt.bar(algorithms, times, color="orange")
plt.ylabel('Tempo de Execução (s)')
plt.xlabel('Algoritmo')
plt.title("Algoritmo x Tempo de Execução")
plt.show()

In [ ]:
# Geramos o gráfico Algoritmo x Acurácia
plt.bar(algorithms, accuracies, color="orange")
plt.ylabel('Acurácia (%)')
plt.xlabel('Algoritmo')
plt.title("Algoritmo x Acurácia")
plt.show()

In [ ]:
# Geramos o gráfico Algoritmo x F1-Score
plt.bar(algorithms, f1_scores, color="orange")
plt.ylabel('F1-Score (%)')
plt.xlabel('Algoritmo')
plt.title("Algoritmo x F1-Score")
plt.show()

Em relação aos acertos nos testes da classe "**Patológico**", os algoritmos Random Forest e KNN obteveram um desempenho excelente, com média 99% de precisão na classe nos testes realizados; logo atrás temos o MLP, com uma média de 89% de precisão, e por último o SVM, com média de 87%.

A análise desta classe foi levada em consideração na análise por se tratar de um ponto crítico, já que uma classificação incorreta na mesma pode causar complicações sérias, dado o problema abordado.

Se tratando de f1-score, os algoritmos Random Forest e KNN obteveram um desempenho excelente, com uma medida de precisão de teste de 100%; logo atrás temos o MLP, com uma média de 97% de precisão, e por último o SVM, com 87%.

# Conclusão

Por fim, podemos concluir que o algoritmo que melhor performou em nossa análise, ficando na primeira colocação em todos os testes, exceto o tempo de execução, foi o Random Forest, que conseguiu aliar uma rápida excução com uma excelente precisão. Pode-se inferir também, com base nos resultados, que o KNN é uma ótima solução também para a classificação do dataset. Ao final, conseguimos notar que o MLP e SVM, apesar do bom desempenho, para o problema em questão, dado os parâmetros utilizados, não são a melhor opção para problemas similares ao estudado nesta análise.

*Feito por Thiago Henrique Leite da Silva, aluno do 5º semestre de Ciência da Computação na Universidade Federal de São Paulo. (UNIFESP)*